In [1]:
import numpy as np
import torch
from tqdm import tqdm
from torch_geometric.data import DataLoader

from cluster_gnn.models import gnn
from cluster_gnn.data import loader as loadtools

In [2]:
# configure the architecture
FINAL_BIAS = False
DIM_EMBED_EDGE = 64
DIM_EMBED_NODE = 32
ROOT_DIR = '/home/jlc1n20/projects/cluster_gnn/'

In [3]:
# load in the network
device = torch.device('cpu')
model = gnn.Net(dim_embed_edge=DIM_EMBED_EDGE,
                dim_embed_node=DIM_EMBED_NODE,
                final_bias=FINAL_BIAS).to(device)
model.load_state_dict(torch.load(ROOT_DIR + '/models/pos80lr14.pt', map_location=device))
model.eval()

Net(
  (conv1): Interaction(
    (mlp_edge): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
    )
    (mlp_node): Sequential(
      (0): Linear(in_features=68, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
  )
  (conv2): Interaction(
    (mlp_edge): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
    )
    (mlp_node): Sequential(
      (0): Linear(in_features=96, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
  )
  (conv3): Interaction(
    (mlp_edge): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
    )
    (mlp_node): Sequential(
      

In [4]:
# load an iterator over the data
dataset = loadtools.EventDataset()
dataset = dataset[90000:90010]
loader = DataLoader(dataset, batch_size=1)

In [7]:
THRESH = 0.90
# loop through the data in evaluation mode, printing classifications
with torch.no_grad():
    for data in tqdm(loader):
#         label = data.y.detach().numpy().astype(bool)
        wiggle = data.y.view(-1, 1)
        break
#         num_pos = int(label.sum())
#         num_neg = len(label) - num_pos
#         output = torch.sigmoid(model(data))
#         pred = np.where(output.detach().numpy() < THRESH, 0, 1).astype(bool).reshape(-1)
#         key, val = np.unique(pred, return_counts=True)
#         classes = {key[0]: val[0], key[1]: val[1]}
#         true_pos = np.bitwise_and.reduce([pred, label], axis=0).sum()
#         true_neg = np.bitwise_and.reduce([pred == False, label == False], axis=0).sum()
#         false_pos = classes[True] - true_pos
#         false_neg = classes[False] - true_neg
#         print(f'Graph has {data.num_edges} edges, of which {num_pos} are positive, and {num_neg} are negative.')
#         print(f'Positive labels: {true_pos} correctly identified, {false_neg} incorrectly marked as negative.')
#         print(f'Negative labels: {true_neg} correctly identified, {false_pos} incorrectly marked as positive.')

  0%|          | 0/10 [00:00<?, ?it/s]


In [6]:
# mean = 0.0
# with torch.no_grad():
#     for data in tqdm(loader):
#         mean += float(len(data.y) - data.y.sum()) / float(data.y.sum())
        
# mean = mean / float(len(loader))
# print(mean)

In [10]:
wiggle.int()

tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]], dtype=torch.int32)